<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module05/03_reducer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: Reducciones espaciales y temporales en colecciones de imágenes**

En esta lectura, aprenderemos sobre:

- Descripción general de los objetos ee.Reducer()
- Reducciones en ImageCollection.
- Reducciones en imagenes.
- Estadisticas regionales de imagenes
- Estadisticas de regiones de imagenes

### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **3) Descripcion general de los objetos ee.Reducer()**

Los reductores (**ee.Reducer()**) son la forma de agregar datos a traves del tiempo, el espacio, las bandas de una imagen, las dimensiones de una matriz y otras estructuras de datos en Earth Engine. La clase **ee.Reducer** especifica como se agregan los datos. Los reductores de esta clase pueden especificar una estadistica simple para usar para la agregacion (por ejemplo, minimo, maximo, media, mediana, desviacion estandar, etc.) o un resumen mas complejo de los datos de entrada (por ejemplo, histograma, regresión lineal, lista). Se pueden producir reducciones sobre:

- **time** = `imageCollection.reduce()`
- **space** = `image.reduceRegion()` `and image.reduceNeighborhood()`
- **bands** = `image.reduce()`

O el espacio de atributos de un FeatureCollection (`featureCollection.reduceColumns()` o los metodos de FeatureCollection que comienzan con `aggregate_`).

<center>
<img src='https://user-images.githubusercontent.com/16768318/73318354-0997de80-4231-11ea-9398-26183673343b.png'>
</center>

In [0]:
# Load and filter the Sentinel-2 image collection.
collection = ee.ImageCollection('COPERNICUS/S2')\
               .filterDate('2016-01-01', '2016-12-31')\
               .filterBounds(ee.Geometry.Point([-81.31, 29.90]))

# Reduce the collection.
extreme = collection.reduce(ee.Reducer.median())
print("Tamaño de collection: ", collection.size().getInfo())
print("Tipo de ED de extreme:",type(extreme))
print("Nombre de las bandas:",extreme.bandNames().getInfo())
print("------")
extreme_extreme = extreme.reduce(ee.Reducer.mean())
print("Tipo de ED de extreme_extreme:",type(extreme))
print("Nombre de las bandas:",extreme_extreme.bandNames().getInfo())


Tamaño de collection:  28
Tipo de ED de extreme: <class 'ee.image.Image'>
Nombre de las bandas: ['B1_median', 'B2_median', 'B3_median', 'B4_median', 'B5_median', 'B6_median', 'B7_median', 'B8_median', 'B8A_median', 'B9_median', 'B10_median', 'B11_median', 'B12_median', 'QA10_median', 'QA20_median', 'QA60_median']
------
Tipo de ED de extreme_extreme: <class 'ee.image.Image'>
Nombre de las bandas: ['mean']


Esto producirá una salida con el **doble del numero de bandas de las entradas**, donde los nombres de banda en la salida tienen "min" o "max" agregado al nombre de la banda.

### **4) Reduccion en un ee.ImageCollection**

Por ejemplo, consideremos que necesitamos tomar el estado medio (e.g la mediana)  de una coleccion temporal de imagenes (representadas en un ee.ImageCollection). Para reducir el ee.ImageCollection, utilize **imageCollection.reduce()**. Esto reduce la coleccion de imagenes a una imagen individual como se ilustra en la Figura en esta celda. Especificamente, la salida se calcula por pixeles, de manera que cada pixel de la salida esta compuesto por el valor medio de todas las imagenes de la coleccion en esa ubicacion. Para obtener otras estadísticas, como la media, la suma, la varianza, un percentil arbitrario, etc., se debe seleccionar y aplicar el reductor apropiado. Para las estadisticas basicas como min, max, media, etc., **ImageCollection** tiene metodos abreviados como min(), max(), median(), etc. Funcionan exactamente de la misma manera que cuando se llama a **reduce()**, excepto que los nombres de las bandas resultantes no tendran el nombre del reductor adjunto.

<center>
<img src='https://user-images.githubusercontent.com/16768318/73318354-0997de80-4231-11ea-9398-26183673343b.png'>
</center>

In [0]:
# Cargue una colección de imágenes, filtrada para que no sean demasiados datos.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1')\
               .filterDate('2008-01-01', '2008-12-31')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# Calcular la mediana en cada banda, cada píxel.
# Los nombres de las bandas son B1_median, B2_median, etc.
median = collection.reduce(ee.Reducer.mean())
print(collection.first().bandNames().getInfo())
print(median.bandNames().getInfo())
# La salida es una imagen. Añádelo al mapa.
vis_param = {'bands': ['B4_mean', 'B3_mean', 'B2_mean'], 'gamma': 1.6}
median_tk = median.getMapId(vis_param)

center = [-122.3355, 37.7924]
mapdisplay(center,{'Landsat 5':median_tk},zoom_start=9)

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BQA']
['B1_mean', 'B2_mean', 'B3_mean', 'B4_mean', 'B5_mean', 'B6_mean', 'B7_mean', 'BQA_mean']


Esto devuelve una imagen multibanda, donde cada pixel representa la mediana de todos los pixeles del ImageCollection. Específicamente, el reductor se ha repetido para cada pixel de las imagenes de entrada. Tenga en cuenta que los nombres de las bandas tienen el nombre del reductor adjunto: "B1_median", "B2_median", etc.

### **5) Reduccion en ee.Image**

Para reducir una imagen, tambien puede usar **image.reduce()**. La reduccion de una imagen funciona de manera analoga a **imageCollection.reduce()**, excepto que las bandas de la imagen se ingresan al reductor en lugar de las imagenes de la coleccion. La salida tambien es una imagen con un numero de bandas igual al numero de salidas del reductor. Por ejemplo:


In [0]:
# Cargue una imagen y seleccione algunas bandas de interés.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')\
          .select(['B4', 'B3', 'B2'])

# Reduzca la imagen para obtener una imagen de valor máximo de una banda.
maxValue = image.reduce(ee.Reducer.max())

# Mostrar el resultado
vis_param = {'max': 13000, 'gamma': 1.6}
center = [-122.3355, 37.7924]

maxValue_tk = maxValue.getMapId(vis_param)
mapdisplay(center,{'Landsat 5':maxValue_tk},zoom_start=10)

### **5) Estadisticas en una ee.Image considerando un dominio espacial - I**

Supongamos que es necesario calcular estadisticas sobre una region (o regiones) de un ee.Image. Para obtener estadisticas de los valores de pixeles en una region de imagen, use **image.reduceRegion()**. Esto reduce todos los pixeles de las regiones a una representacion estadistica u otra representacion compacta de los datos de pixeles en la region (por ejemplo, histograma). La region se representa como un **ee.Geometry**, que podria ser un poligono, que contiene muchos pixeles, o podria ser un solo punto, en cuyo caso solo habra un pixel en la region. En cualquier caso, como se ilustra en la figura de abajo, la salida es una estadistica derivada de los pixeles en la region.

<center>
<img src='https://user-images.githubusercontent.com/16768318/73318368-17e5fa80-4231-11ea-86dd-db5e71715d23.png'>
</center>

In [0]:
from pprint import pprint

# Cargar imágenes de entrada: Landsat 7 compuesto de 5 años.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Cargar una region de entrada: bosque de coniferas mixtas de Sierra Nevada.
region = ee.Feature(ee.FeatureCollection('EPA/Ecoregions/2013/L3')
  .filter(ee.Filter.eq('us_l3name', 'Sierra Nevada'))\
  .first())

# Reducir la region. El parametro de la region es Geometria de entidades.
meanDictionary = image.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = region.geometry(),
  scale = 30,
  maxPixels = 10**9
)

# El resultado es un diccionario. Imprimelo.
pprint(meanDictionary.getInfo())

{'B1': 25.40602971681686,
 'B2': 23.97149701423899,
 'B3': 22.91059593763104,
 'B4': 54.831641332934055,
 'B5': 38.076554725736784,
 'B6_VCID_2': 198.93216428012914,
 'B7': 24.06326163496157}


Tenga en cuenta que en este ejemplo la reduccion se especifica al proporcionar el reductor (`ee.Reducer.mean()`), la geometría (`region.geometry()`), la escala (30 metros) y **maxPixels** para la cantidad maxima de pixeles para ingresar al reductor. Siempre se debe especificar una escala en las llamadas a `reduceRegion()`. Esto se debe a que en los flujos de procesamiento se puede involucrar datos de diferentes fuentes con diferentes escalas, la escala de la salida no se determinara inequivocamente a partir de los datos de entrada. En ese caso, **la escala por defecto es de 1 grado**, lo que generalmente produce resultados insatisfactorios. Consulte esta pagina para obtener mas informacion sobre como Earth Engine maneja la [escala](https://developers.google.com/earth-engine/scale).

### **6) Estadisticas en una ee.Image considerando un dominio espacial - II**

Para obtener estadisticas de imagenes en multiples regiones almacenadas en una `FeatureCollection`, puede usar **image.reduceRegions()** para reducir multiples regiones a la vez. La entrada para ***.reduceRegions()** es una imagen y una función `FeatureCollection`. La salida es otra `FeatureCollection` con la salida de ***.reduceRegions()** establecida como propiedades en cada `Feature`. En este ejemplo, los compuestos anuales de Landsat 7 en cada geometria de entidad se agregaran como propiedades a las entidades (features) de entrada:

In [0]:
# Cargar imágenes de entrada: Landsat 7 compuesto de 5 años.
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/2008_2012')

# Cargar un FeatureCollection
maineCounties = ee.FeatureCollection('TIGER/2016/Counties')\
                  .filter(ee.Filter.eq('STATEFP', '23'))

# Agregue la salida del reductor al FeatureCollection especificado.
maineMeansFeatures = image.reduceRegions(
  reducer = ee.Reducer.mean(),
  collection = maineCounties,
  scale = 30
)

# Imprima el primer feature, para ilustrar el resultado.
feature_first = ee.Feature(maineMeansFeatures.first())
pprint(feature_first.toDictionary().getInfo())
pprint(maineCounties.first().toDictionary().getInfo())

#### **7) Reduccion de las propiedades de un ee.FeatureCollection o ee.ImagenCollection**

In [0]:
from pprint import pprint 

# Cree un FeatureCollection de ejemplo.
aFeatureCollection = ee.FeatureCollection([
  ee.Feature(None, {'foo': 1, 'weight': 1}),
  ee.Feature(None, {'foo': 2, 'weight': 2}),
  ee.Feature(None, {'foo': 3, 'weight': 3}),
])

# Calcule una media ponderada y muestrela.
pprint(aFeatureCollection.reduceColumns(
  reducer = ee.Reducer.mean(),
  selectors = ['foo'],
  weightSelectors = ['weight']
  ).getInfo())

{'mean': 2.3333333333333335}


### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>